In [2]:
### This will look for all the autoTMA images, and check if the corresponding cores are on the server
### if not, generate and upload them
## It also generate a composite core of the H&E and the TMA core
import dsaSecrets, io
import girder_client, os
%load_ext autoreload
%autoreload 2
import tmaHelpers as tma

gc = girder_client.GirderClient(apiUrl=dsaSecrets.apiUrl)
gc.authenticate(apiKey=dsaSecrets.apiToken)
## Samples Images 62bb03e4e70def4ae72689f5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


{'_id': '5cacfa86e62914004a60e139'}

In [3]:
tmaCoreInfo = {}  ### I'm going to stuck all of the information for all of the TMA slides into this dictionary for processing
### This will grab all the ['B7']
studyImageId = '62ded6cbe70def4ae726b228'
studyImages = list(gc.listItem(studyImageId))
for s in studyImages:
    autoTMAdata = tma.getAutoTMAcoreInfo(gc,s['_id'])
    tmaCoreInfo[s['name']] = {'itemInfo': s, 'autoTMAdata': autoTMAdata}

1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
45 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
45 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export
1 autoTMA annotations found... will use the most recent one
45 total ROIs to export
1 autoTMA annotations found... will use the most recent one
150 total ROIs to export


In [4]:
### get information for the TMA cores that are on the server..
tmaCorePath = '/collection/Badve_TMA_Project/StudyImages/autoTMAcores'
tcp = gc.get('resource/lookup?path=%s' % tmaCorePath)

tmaCoreImages = list(gc.listItem(tcp['_id']))
print(len(tmaCoreImages),'TMA Cores have been uploaded')

uploadedTmaCores = [x['name'] for x in tmaCoreImages]

1185 TMA Cores have been uploaded


In [6]:
# export_tma_cores( gc, imgId, roiUploadDir, tmaAnnotationName='autoTMA'):
roiNameSet = []

for i in studyImages:
    print("Processing",i['name'])
    roiNameSet.extend( tma.export_tma_cores(gc,i['_id'],uploadedTmaCores) )

Processing 254461.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 254462.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 257591.svs
1 autoTMA annotations found... will use the most recent one and
45 total ROIs to export
Processing 257593.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 257595.svs
1 autoTMA annotations found... will use the most recent one and
45 total ROIs to export
Processing 257597.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 257600.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 257601.svs
1 autoTMA annotations found... will use the most recent one and
150 total ROIs to export
Processing 257603.svs
1 autoTMA annotations found... will use the most recent one and
45 total ROIs to export
Proc

In [7]:
for x in tmaCoreImages:
    if x['name'] not in roiNameSet:
        print(x['name'],x['_id'])
        gc.delete('item/%s' % x['_id'])

In [ ]:
# yamlFolder = '62e33fcde70def4ae726fcb7'
# f = io.StringIO(yaml.dump(members))

# gc.uploadFile(yamlFolder,f,'hellomoto.yaml',70,parentType='folder')

In [12]:
def writeMultiCoreYaml( gc, imgIdOne, imgIdTwo):
    ### This will take two images and concanetate them together with imgOne on the left and image two on the right
    ### Using the nifty new YAML configuration...
    ### Get the tileInfo for these two slides..
    imgOneTs = gc.get('item/%s/tiles' % imgIdOne)
    imgTwoTs = gc.get('item/%s/tiles' % imgIdTwo)
#     print(imgOneTs)
    mcWidth = imgOneTs['sizeX'] + imgTwoTs['sizeX']
    mcHeight = max([imgOneTs['sizeX'],imgTwoTs['sizeX']])
    
    return (
    f'---\r\n'
    f'width: {mcWidth}\r\n'
    f'height: {mcHeight}\r\n'
    'sources:\r\n'
    f'  - path: girder://{imgIdOne}\r\n'
    f'    z: 0\r\n'
    f'    position:\r\n'
    f'      x: 0\r\n'
    f'      y: 0\r\n'
    f'  - path: girder://{imgIdTwo}\r\n'
    f'    z: 0\r\n'
    f'    position:\r\n'
    f'      x: {imgOneTs["sizeX"]}\r\n'
    f'      y: 0\r\n'
    )

In [13]:
### Generate the pairs of images for the multicore yaml file..
## An HE image should be paired with the corresponding 
### The H&E images are 17 and 18 
### These need to be mapped
HEslideDict = {}

for c in tmaCoreImages:
    (slide,coreId,cx,cy,ext) = c['name'].split(".")
    if slide in ['S17','S18']:
        if coreId not in HEslideDict:
            HEslideDict[coreId] = c
        else:
            print("Duplicate HE core detected for",coreId,slide)

## Now that I have the HEs to pair, I Can loop through all the cores and generate the composite

yamlFolder = '62e33fcde70def4ae726fcb7'
# f = io.StringIO(yaml.dump(members))
# gc.uploadFile(yamlFolder,f,'hellomoto.yaml',70,parentType='folder')

## GEt a list of all the existing YAML files in the target directory
curYamlFileList = [x['name'] for x in list(gc.listItem(yamlFolder))]
for c in tmaCoreImages:
    (slide,coreId,cx,cy,ext) = c['name'].split(".")
    if slide  not in ['S17','S18']:
        ## So these are then.. NOT H&E cores
        yamlFileName = f'{slide}.{coreId}.yaml'
        if yamlFileName not in curYamlFileList:
        
            try:
                mcYaml = writeMultiCoreYaml(gc, c['_id'], HEslideDict[coreId]['_id'] )
                f = io.StringIO(mcYaml)
                gc.uploadFile(yamlFolder,f, yamlFileName,len(mcYaml),parentType='folder')
            except:
                print("Some error with",slide,coreId)

In [ ]:
HEslideDict.keys()

In [ ]:
c

In [ ]:
# import yaml
# x = yaml.dump(members)
# len(x)

len(yaml.dump(members))
# # dict object
# members = [{'name': 'Zoey', 'occupation': 'Doctor'},
#            {'name': 'Zaara', 'occupation': 'Dentist'}]

# # Convert Python dictionary into a YAML document
# print(yaml.dump(members))

In [ ]:
tmaCoreImages[2]

In [ ]:
f=writeMultiCoreYaml(gc,tmaCoreImages[1]['_id'],tmaCoreImages[2]['_id'])
print(f)

In [ ]:
# def getAutoTMAcoreInfo( gc, imgId, tmaAnnotationName='autoTMA'):
#     ### This will find and return the latest autoTMA record associated with a given image
#     i = gc.get('item/%s' % imgId)
#     fileRoot = i['meta']['slideNumber']
    
#     ## Get autoTMA documents for this item
#     tmaDocs=  gc.get('annotation?itemId=%s&name=%s' % (imgId, tmaAnnotationName))
#     print(len(tmaDocs),'autoTMA annotations found... will use the most recent one')
    
#     if tmaDocs:
#         ae = gc.get('annotation/%s' % tmaDocs[-1]['_id']) ## Grab the last one... assume that's the most recent.. may need to check
#         print(len(ae['annotation']['elements']),'total ROIs to export')
#     else:
#         ae = None
#         print("No tma cores found for this image..")
    
#     return ae
#     roiDir = 'ROI_Export'
    
#     if not os.path.isdir(roiDir):
#         os.mkdir(roiDir)

#     for roi in ae['annotation']['elements']:
#         roiFileName = "%s/%s.%s.%d.%d.tiff" % (roiDir,fileRoot,roi['label']['value'],roi['center'][0],roi['center'][1])
        
#         if not os.path.isfile(roiFileName):
#             print(roiFileName)
#             r= tma.get_annotation_region(gc,imgId,roi)        

#             with open(roiFileName,"wb") as fpi:
#                 fpi.write(r)
        
       